In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from random import shuffle
from sklearn.linear_model import SGDClassifier


pd.set_option('display.max_colwidth', -1)

In [22]:
train=pd.read_csv('train.csv')
#train.set_index('Webpage_id',inplace=True)
test=pd.read_csv('test_nvPHrOx.csv')
#test.set_index('Webpage_id',inplace=True)
submission=pd.read_csv('sample_submission_poy1UIu.csv')

In [23]:
def preprocess(url):
    
    pattern=r'//.*'
    urls=[]
    for i in url:
        b=re.findall(pattern,i)
        c=b[0][2:]
        d=re.split('\.|/|-|_',c)
        e=''
        for i in d:
            e+=str(i).lower()
            e+=' '
        e = ''.join([i for i in e if not i.isdigit()])
        urls.append(e)
        
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(urls)
    
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

    
    
    
    return(urls)

In [24]:
def train_val_split(train):
    grp=train.groupby('Tag')
    
    test_set=[]
    train_set=[]
    test_tag_set=[]
    train_tag_set=[]
    count=0
    for unique_tags in train.Tag.unique():
        tag=grp.get_group(unique_tags)
        s=set(tag.Domain)
        count+=len(s)
        a=list(s)
        #shuffle(a)
        for i in range(len(a)):
            if (i+1)%3==0:
                test_set.append(a[i])
                test_tag_set.append(unique_tags)
            else:
                train_set.append(a[i])
                train_tag_set.append(unique_tags)
    
    
    train_domain=pd.DataFrame(train_set)
    train_domain.columns=['Domain']
    train_domain['Tag']=train_tag_set
    train_data=pd.merge(train_domain,train,on=['Domain','Tag'],how='left')
    
    val_domain=pd.DataFrame(test_set)
    val_domain.columns=['Domain']
    val_domain['Tag']=test_tag_set
    val_data=pd.merge(val_domain,train,on=['Domain','Tag'],how='left')
    
    return(train_data,val_data)
    

In [25]:
train_data,val_data=train_val_split(train)

In [26]:
train_url=preprocess(train_data.Url)
train_target=train_data.Tag.values

In [27]:
val_url=preprocess(val_data.Url)
#val_data.drop('Tag',axis=1,inplace=True)
val_target=val_data.Tag.values

# MultinomialNB

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(train_url, train_target)

In [17]:
predicted = text_clf.predict(train_url)
print('train_score')
print(np.mean(predicted == train_target))

predicted = text_clf.predict(val_url)
print('val_score')
np.mean(predicted == val_target)

train_score
0.9314739053516143
val_score


0.5986335475652829

# SVM

In [28]:
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(train_url, train_target)


C:\Users\Nishant\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [29]:
predicted = text_clf_svm.predict(train_url)
print('train_score')
print(np.mean(predicted == train_target))

predicted = text_clf_svm.predict(val_url)
print('val_score')
np.mean(predicted == val_target)

train_score
0.91187527642636
val_score


0.7115974755370273

In [ ]:
train_score
0.9127874834144184
val_score
0.6868739505529501